## Another Approach

In [1]:
def exhaustiveChange(amount, denominations):
    bestN = 100
    count = [0 for i in range(len(denominations))]
    while True:
        for i, coinValue in enumerate(denominations):
            count[i] += 1
            if (count[i]*coinValue < 100):
                break
            count[i] = 0
        n = sum(count)
        if n == 0:
            break
        value = sum([count[i]*denominations[i] for i in range(len(denominations))])
        if (value == amount):
            if (n < bestN):
                solution = [count[i] for i in range(len(denominations))]
                bestN = n
    return solution

print(exhaustiveChange(40,[1,5,10,20,25]))

[0, 0, 0, 2, 0]


## Correct, but costly
* Our algorithm now gets the right answer for every value 1..100
* It must, because it considers every possible answer<br>(that’s the good thing about brute force)
* There is a downside though

In [2]:
%time print(exhaustiveChange(40, [1,5,10,25]))
%time print(exhaustiveChange(40, [1,5,10,20,25]))
%time print(exhaustiveChange(40, [1,3,5,7,11,13]))

[0, 1, 1, 1]
CPU times: user 142 ms, sys: 3.78 ms, total: 146 ms
Wall time: 144 ms
[0, 0, 0, 2, 0]
CPU times: user 754 ms, sys: 0 ns, total: 754 ms
Wall time: 754 ms
[0, 0, 0, 1, 3, 0]
CPU times: user 2min 40s, sys: 23.5 ms, total: 2min 40s
Wall time: 2min 40s


## Other tricks?

A Branch-and-bound algorithm

In [3]:
def branchAndBoundChange(amount, denominations):
    bestN = amount
    count = [0 for i in range(len(denominations))]
    while True:
        for i, coinValue in enumerate(denominations):
            count[i] += 1
            if (count[i]*coinValue < amount):             # Set upper bound to amount rather than 100
                break
            count[i] = 0
        n = sum(count)
        if n == 0:
            break
        if (n > bestN):                                   # don't compute the amount if there are too many coins
            continue
        value = sum([count[i]*denominations[i] for i in range(len(denominations))])
        if (value == amount):
            if (n < bestN):
                solution = [count[i] for i in range(len(denominations))]
                bestN = n
    return solution

%time print(branchAndBoundChange(40, [1,3,5,7,11,13]))

[0, 0, 0, 1, 3, 0]
CPU times: user 281 ms, sys: 1 µs, total: 281 ms
Wall time: 279 ms


## A Recursive Coin-Change Algorithm

In [4]:
def RecursiveChange(M, c):
    if (M == 0):
        return [0 for i in range(len(c))]
    smallestNumberOfCoins = M+1
    for i in range(len(c)):
        if (M >= c[i]):
            thisChange = RecursiveChange(M - c[i], c)
            thisChange[i] += 1
            if (sum(thisChange) < smallestNumberOfCoins):
                bestChange = thisChange
                smallestNumberOfCoins = sum(thisChange)
    return bestChange

%time print(RecursiveChange(40, [1,3,5,7,11,13]))

[1, 0, 0, 0, 0, 3]
CPU times: user 4min 45s, sys: 18.1 ms, total: 4min 45s
Wall time: 4min 45s


## Change via Dynamic Programming

In [15]:
def DPChange(M, c):
    change = [[0 for i in range(len(c))]]
    for m in range(1,M+1):
        bestNumCoins = m+1
        for i in range(len(c)):
            if (m >= c[i]):
                thisChange = [x for x in change[m - c[i]]]
                thisChange[i] += 1
                if (sum(thisChange) < bestNumCoins):
                    change[m:m] = [thisChange]
                    bestNumCoins = sum(thisChange)
    return change[M]

%time print(DPChange(40, [1,3,5,7,11,13]))
%time print(DPChange(40, [1,3,5,7,11,13,17]))
%time print(DPChange(40, [1,3,5,7,11,13,17,19]))

[1, 0, 0, 0, 0, 3]
CPU times: user 692 µs, sys: 0 ns, total: 692 µs
Wall time: 540 µs
[1, 0, 1, 0, 0, 0, 2]
CPU times: user 332 µs, sys: 0 ns, total: 332 µs
Wall time: 329 µs
[2, 0, 0, 0, 0, 0, 0, 2]
CPU times: user 350 µs, sys: 0 ns, total: 350 µs
Wall time: 335 µs


## A Hybrid Approach: Memoization

In [34]:
change = {}                                            # This is a cache for saving bestChange[M]

def MemoizedChange(M, c):
    global change
    if (M in change):                                   # Check the cache first
        return [v for v in change[M]]
    if (len(change) == 0):                              # Initialize cache
        change[0] = [0 for i in range(len(c))]
    smallestNumberOfCoins = M+1
    for i in range(len(c)):
        if (M >= c[i]):
            thisChange = MemoizedChange(M - c[i], c)
            thisChange[i] += 1
            if (sum(thisChange) < smallestNumberOfCoins):
                bestChange = [v for v in thisChange]
                smallestNumberOfCoins = sum(thisChange)
    change[M] = [v for v in bestChange]                 # Add new M to cache 
    return bestChange

%time print(MemoizedChange(40, [1,3,5,7,11,13]))

[1, 0, 0, 0, 0, 3]
CPU times: user 541 µs, sys: 0 ns, total: 541 µs
Wall time: 477 µs
